### Importing libraries

In [59]:
import pandas as pd
import numpy as np
import gc
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

### Reading the datasets

In [60]:
df = pd.read_csv('transformed_structured_data.csv', index_col=0)
df.head()

,Rank,Rank Opp.,Int. Age,Opp. Age,Class,Sport_MMA,Sport_Tennis,Interviewee_Andy Mury,Interviewee_Andy Ruiz,Interviewee_Anita Doganova,Interviewee_Anthony Djoshua,Interviewee_Blagoi Ivanov,Interviewee_Bogdan Dinu,Interviewee_Cvetana Pironkova,Interviewee_Daniel Zlatkov,Interviewee_David Gofen,Interviewee_Dereck Chisora,Interviewee_Deyan Topalski,Interviewee_Dominic Thiem,Interviewee_Fabio Fonini,Interviewee_Grigor Dimitrov,Interviewee_Hughie Fury,Interviewee_Kevin Johnson,Interviewee_Kubrat Pulev,Interviewee_Maria Sharapova,Interviewee_Mario Cilic,Interviewee_Nikolas Basilashvili,Interviewee_Novak Djokovic,Interviewee_Rafael Nadal,Interviewee_Roger Federer,Interviewee_Stan Wawrinka,Interviewee_Stefanos Cicipas,Interviewee_Svetlozar Savov,Interviewee_Tervel Pulev,Interviewee_Tommy Robredo,Interviewee_Tsvetozar Iliev,Interviewee_Tyson Fury,Interviewee_Wladimir Klitschko,Opponent_Aleksandra Toncheva,Opponent_Andy Ruiz,Opponent_Anthony Djoshua,Opponent_Atanas Mihaylov,Opponent_Ben Rothwell,Opponent_Bogdan Dinu,Opponent_Caroline Wozniacki,Opponent_Cvetana Pironkova,Opponent_David Gofen,Opponent_Dereck Chisora,Opponent_Dominic Thiem,Opponent_Francesco Pianeta,Opponent_Gael Montfis,Opponent_Grigor Dimitrov,Opponent_Grigor Saruhanian,Opponent_Hughie Fury,Opponent_Jerzy Janowicz,Opponent_Kevin Johnson,Opponent_Kubrat Pulev,Opponent_Leonardo Bruzzese,Opponent_Marcos Bagdatis,Opponent_Maurice Harris,Opponent_Muslim Salihov,Opponent_Nikolas Basilashvili,Opponent_Piotr Strus,Opponent_Rafael Nadal,Opponent_Roger Federer,Opponent_Stan Wawrinka,Opponent_Tommy Robredo,Opponent_Tony Thompson,Opponent_Viktor Troicky,Opponent_Wladimir Klitschko,Health_H,Health_S,Psychics_3,Psychics_4,Psychics_5,Prev. Match_N,Prev. Match_W,Prev. Match_Y,Confidence_2,Confidence_3,Confidence_4,Confidence_5,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,dayofyear_13,dayofyear_25,dayofyear_27,dayofyear_39,dayofyear_42,dayofyear_64,dayofyear_82,dayofyear_85,dayofyear_108,dayofyear_118,dayofyear_119,dayofyear_132,dayofyear_133,dayofyear_149,dayofyear_151,dayofyear_180,dayofyear_187,dayofyear_218,dayofyear_230,dayofyear_236,dayofyear_245,dayofyear_246,dayofyear_247,dayofyear_265,dayofyear_275,dayofyear_278,dayofyear_280,dayofyear_300,dayofyear_306,dayofyear_319,dayofyear_320,dayofyear_332,week_2,week_4,week_6,week_7,week_10,week_12,week_16,week_17,week_20,week_22,week_26,week_27,week_32,week_33,week_34,week_35,week_36,week_38,week_40,week_43,week_44,week_46,week_48,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,is_male_1,is_int_prime_age_1,is_opp_prime_age_1,age_gap_0.2631578947368421,age_gap_0.2894736842105263,age_gap_0.3157894736842105,age_gap_0.3421052631578947,age_gap_0.39473684210526316,age_gap_0.47368421052631576,age_gap_0.5,age_gap_0.5263157894736842,age_gap_0.5526315789473684,age_gap_0.5789473684210527,age_gap_0.6052631578947368,age_gap_0.631578947368421,age_gap_0.6578947368421052,age_gap_0.6842105263157894,age_gap_0.7105263157894737,age_gap_0.7368421052631579,age_gap_0.763157894736842,age_gap_0.7894736842105263,age_gap_0.8684210526315789,age_gap_0.9210526315789473,age_gap_0.9736842105263157,age_gap_1.0
0,0.125461,0.774908,0.948718,0.409091,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0.118081,0.811808,0.692308,0.454545,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0.774908,0.125461,0.666667,0.909091,1,0,0,0,0,0,1,0,0

In [61]:
application = df.copy()

### Do sampling if needed

In [62]:
categorical_list = []
numerical_list = []
for i in application.columns.tolist():
    if application[i].dtype=='object':
        categorical_list.append(i)
    else:
        numerical_list.append(i)
print('Number of categorical features:', str(len(categorical_list)))
print('Number of numerical features:', str(len(numerical_list)))

Number of categorical features: 0
Number of numerical features: 176


Feature matrix and target

In [64]:
X = application.drop(['Class'], axis=1)
y = application['Class']
feature_name = X.columns.tolist()

### Feature Selection
- ***xxx_support***: list to represent select this feature or not
- ***xxx_feature***: the name of selected features

### 1 Filter

#### 1.1 Pearson Correlation

___Note___
- Normalization: no
- Impute missing values: yes

In [65]:
def cor_selector(X, y):
    cor_list = []
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-100:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature

In [66]:
cor_support, cor_feature = cor_selector(X, y)
print(str(len(cor_feature)), 'selected features')

100 selected features


#### 1.2 Chi-2
___Note___
- Normalization: MinMaxScaler (values should be bigger than 0)
- Impute missing values: yes

In [67]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=10)
chi_selector.fit(X_norm, y)

SelectKBest(k=10, score_func=<function chi2 at 0x000002A3C8BB8678>)

In [68]:
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

10 selected features


### 2 Wrapper

___Note___
- Normalization: depend on the used model; yes for LR
- Impute missing values: depend on the used model; yes for LR

In [69]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10, verbose=5)
rfe_selector.fit(X_norm, y)

Fitting estimator with 175 features.
Fitting estimator with 165 features.
Fitting estimator with 155 features.
Fitting estimator with 145 features.
Fitting estimator with 135 features.
Fitting estimator with 125 features.
Fitting estimator with 115 features.
Fitting estimator with 105 features.


RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=100, step=10, verbose=5)

In [70]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

100 selected features


### 3. Embeded

#### 3.1 Logistics Regression L2
___Note___
- Normalization: Yes
- Impute missing values: Yes

In [71]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), '1.25*median')
embeded_lr_selector.fit(X_norm, y)

SelectFromModel(estimator=LogisticRegression(C=1.0, class_weight=None,
                                             dual=False, fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l2',
                                             random_state=None, solver='lbfgs',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False,
                threshold='1.25*median')

In [72]:
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

73 selected features


#### 3.2 Random Forest
___Note___
- Normalization: No
- Impute missing values: Yes

In [73]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), threshold='1.25*median')
embeded_rf_selector.fit(X, y)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=None,
                                                 oob_score=False,

In [74]:
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

68 selected features


#### 3.3 LightGBM
___Note___
- Normalization: No
- Impute missing values: No

In [75]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, threshold='1.25*median')
embeded_lgb_selector.fit(X, y)

SelectFromModel(estimator=LGBMClassifier(boosting_type='gbdt',
                                         class_weight=None,
                                         colsample_bytree=0.2,
                                         importance_type='split',
                                         learning_rate=0.05, max_depth=-1,
                                         min_child_samples=20,
                                         min_child_weight=40,
                                         min_split_gain=0.01, n_estimators=500,
                                         n_jobs=-1, num_leaves=32,
                                         objective=None, random_state=None,
                                         reg_alpha=3, reg_lambda=1, silent=True,
                                         subsample=1.0,
                                         subsample_for_bin=200000,
                                         subsample_freq=0),
                max_features=None, norm_order=1, prefit=False

In [76]:
embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

175 selected features


### Summary

In [77]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(100)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,year_2015,True,True,True,True,True,True,6
2,age_gap_0.763157894736842,True,True,True,True,True,True,6
3,age_gap_0.6842105263157894,True,True,True,True,True,True,6
4,Prev. Match_W,True,True,True,True,True,True,6
5,Opponent_Kubrat Pulev,True,True,True,True,True,True,6
6,Interviewee_Kubrat Pulev,True,True,True,True,True,True,6
7,Interviewee_Grigor Dimitrov,True,True,True,True,True,True,6
8,Confidence_4,True,True,True,True,True,True,6
9,Confidence_2,True,True,True,True,True,True,6
10,year_2019,True,False,True,True,True,True,5


### Embeded Random Forest Features

In [78]:
embeded_rf_feature

['Rank',
 'Rank Opp.',
 'Int. Age',
 'Opp. Age',
 'Sport_MMA',
 'Sport_Tennis',
 'Interviewee_Andy Ruiz',
 'Interviewee_Anthony Djoshua',
 'Interviewee_Bogdan Dinu',
 'Interviewee_David Gofen',
 'Interviewee_Grigor Dimitrov',
 'Interviewee_Hughie Fury',
 'Interviewee_Kubrat Pulev',
 'Interviewee_Mario Cilic',
 'Interviewee_Nikolas Basilashvili',
 'Interviewee_Svetlozar Savov',
 'Interviewee_Tommy Robredo',
 'Opponent_Andy Ruiz',
 'Opponent_Anthony Djoshua',
 'Opponent_Dominic Thiem',
 'Opponent_Grigor Dimitrov',
 'Opponent_Kubrat Pulev',
 'Opponent_Piotr Strus',
 'Opponent_Roger Federer',
 'Opponent_Stan Wawrinka',
 'Psychics_3',
 'Psychics_4',
 'Psychics_5',
 'Prev. Match_N',
 'Prev. Match_W',
 'Prev. Match_Y',
 'Confidence_2',
 'Confidence_3',
 'Confidence_4',
 'Confidence_5',
 'year_2014',
 'year_2015',
 'year_2018',
 'year_2019',
 'dayofyear_119',
 'dayofyear_149',
 'dayofyear_151',
 'dayofyear_246',
 'dayofyear_278',
 'dayofyear_319',
 'week_4',
 'week_12',
 'week_17',
 'week_22',

### Top Features

In [79]:
top_1_features_df = feature_selection_df.loc[feature_selection_df['Total']==6]
top_1_features = top_1_features_df['Feature']

In [80]:
top_2_features_df = feature_selection_df.loc[feature_selection_df['Total']==5]
top_2_features = top_2_features_df['Feature']

In [81]:
top_1_features

1                      year_2015
2      age_gap_0.763157894736842
3     age_gap_0.6842105263157894
4                  Prev. Match_W
5          Opponent_Kubrat Pulev
6       Interviewee_Kubrat Pulev
7    Interviewee_Grigor Dimitrov
8                   Confidence_4
9                   Confidence_2
Name: Feature, dtype: object

In [82]:
top_2_features

10                           year_2019
11                              week_4
12                  is_int_prime_age_1
13                       dayofyear_246
14                       dayofyear_149
15          age_gap_0.9210526315789473
16          age_gap_0.3421052631578947
17          age_gap_0.2894736842105263
18                          Psychics_4
19                       Prev. Match_Y
20              Opponent_Stan Wawrinka
21              Opponent_Roger Federer
22                Opponent_Piotr Strus
23                  Opponent_Andy Ruiz
24                            Opp. Age
25           Interviewee_Tommy Robredo
26         Interviewee_Svetlozar Savov
27    Interviewee_Nikolas Basilashvili
28             Interviewee_Mario Cilic
29             Interviewee_Hughie Fury
30             Interviewee_David Gofen
31             Interviewee_Bogdan Dinu
32               Interviewee_Andy Ruiz
Name: Feature, dtype: object

### Filters

In [83]:
embeded_rf_feature_class = list(embeded_rf_feature) + ['Class']

In [84]:
top_1_features = list(top_1_features)

In [85]:
top_1_features_class = top_1_features + ['Class']

In [86]:
top_2_features = list(top_2_features)

In [87]:
top_2_features_class = top_2_features + ['Class']

In [88]:
top_1_and_2_features = top_1_features + top_2_features

In [89]:
top_1_and_2_features_class = top_1_and_2_features + ['Class']

### New Dataframes

In [90]:
top_1_features_class

['year_2015',
 'age_gap_0.763157894736842',
 'age_gap_0.6842105263157894',
 'Prev. Match_W',
 'Opponent_Kubrat Pulev',
 'Interviewee_Kubrat Pulev',
 'Interviewee_Grigor Dimitrov',
 'Confidence_4',
 'Confidence_2',
 'Class']

In [91]:
df_top_1 = df[top_1_features_class]
df_top_1_and_2 = df[top_1_and_2_features_class]
df_embeded_rf_feature = df[embeded_rf_feature_class]

### Saving Dataframes

In [93]:
df_top_1.to_csv('df_top_1_structured.csv', sep=',', encoding='utf-8', header=True, index=True)

In [94]:
df_top_1_and_2.to_csv('df_top_1_and_2_structured.csv', sep=',', encoding='utf-8', header=True, index=True)

In [95]:
df_embeded_rf_feature.to_csv('df_embeded_rf_feature.csv', sep=',', encoding='utf-8', header=True, index=True)